In [50]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from src.io_spec import get_dataset_from_template

wavemax = None
wavemin = 4000
n_specs = 100  # Setting a proper value instead of None
planetary_signal_amplitudes = [100.0]
planetary_signal_periods = [10.0]
noise_level = 0
template = np.load(
    "/home/tliopis/Codes/lagrange_llopis_mary_2025/data/soapgpu/paper_dataset/spec.npy"
)
template = template / np.max(template)  # Normalisation du template
wavegrid = np.load(
    "/home/tliopis/Codes/lagrange_llopis_mary_2025/data/soapgpu/paper_dataset/wavegrid.npy"
)

velocities = np.zeros(n_specs, dtype=np.float32)
for amplitude, period in zip(
    planetary_signal_amplitudes, planetary_signal_periods
):
    velocities += amplitude * np.sin(
        2 * np.pi * np.arange(n_specs) / period
    )

dataset, wavegrid, template = get_dataset_from_template(
    template=template,
    wavegrid=wavegrid,
    velocities=velocities,
    wavemin=wavemin,
    wavemax=wavemax,
    noise_level=noise_level,
    dtype=torch.float32,
    verbose=True,
)

Génération du dataset de spectres à partir du modèle... (sur cuda)
Batch traité, taille : 100


In [51]:
%matplotlib qt
plt.figure(figsize=(10, 5))
plt.plot(wavegrid, template, label="Template")
plt.plot(wavegrid, dataset[9], label="Dataset Sample")
plt.xlabel("Wavelength (A)")
plt.ylabel("Flux")
plt.title("Spectral Template and Sample from Dataset")
plt.xlim(5005, 5008)
plt.grid()
plt.legend()
plt.show()

In [52]:
from src.ccf import compute_CCFs_mask_gauss, compute_CCFs_template_optimized
from src.fitter import extract_rv_from_fit, CCFFitter


v_grid = np.arange(-20000, 20000, 100)
window_size_velocity = 820  # Example value, adjust as needed
# Example usage of compute_CCFs_template_optimized
CCFs_mask = compute_CCFs_mask_gauss(
    specs=dataset,
    v_grid=v_grid,
    wavegrid=wavegrid,
    verbose=True,
    mask_type="G2",
    sigma_velocity=window_size_velocity,
)

Construction CCF gaussian sparse...


In [53]:
from src.fitter import CCFFitter, extract_rv_from_fit
from src.ccf import compute_CCFs_template_optimized, compute_CCFs_mask_gauss

fitter = CCFFitter()
rvs = []

for ccf in CCFs_mask:
    res = fitter.fit_ccf(ccf, v_grid=v_grid, orientation="down", model="gaussian", window_size=window_size_velocity)
    rv = extract_rv_from_fit(res)[0]
    rvs.append(rv)

rvs = np.array(rvs)

plt.figure(figsize=(10, 5))
plt.plot(rvs, label="RV from CCFs")
plt.xlabel("Spectrum Index")
plt.ylabel("Radial Velocity (m/s)")
plt.title("Radial Velocities from CCFs")
plt.grid()
plt.legend()
plt.show()